# ACS Labor Force Change 2006-2010 to 2016-2020

#### This notebook analyzes changes to the NYC Metro labor force by age cohort, relative to the base population trend

- https://www.census.gov/data/developers/data-sets/acs-5year.html

For total labor force & population
- https://api.census.gov/data/2010/acs/acs5/profile/variables.html
- https://api.census.gov/data/2020/acs/acs5/profile/variables.html

For labor force by cohort
- https://api.census.gov/data/2010/acs/acs5/variables.html
- https://api.census.gov/data/2020/acs/acs5/variables.html

### Set proxy prior to running

In [1]:
# SET TO TRUE/FALSE
onDCPServer = True
#
#

In [2]:
import pandas as pd
import math
import numpy as np
import os

In [3]:
from geo import *
from make_labfor import *
import make_labfor as lf
import get_acs as get
import geo_agg

import utilcalcs as calc
import geo_agg as geo
from censusAPI import myAPI
from county_codes import stco

In [4]:
## proxy assignment, None if working off-network
if onDCPServer == False:
    p = None
else:
    from proxy import p

In [5]:
date = '220425'

In [6]:
#My search parameters
y1 = '2020'
y0 = '2010'

cols_s = f'group(B23001)'
cols_p = f'GEO_ID,DP03_0002E,DP03_0002M'

source_s = 'acs/acs5'
source_p = 'acs/acs5/profile'

## NYC Metro counties, subregion, region labor force in Y1 and Y0, with change

In [7]:
dfY1_p = get.get_county(p,source_p,y1,cols_p)
dfY1_s = get.get_county(p,source_s,y1,cols_s)
dfY1 = pd.merge(dfY1_p,dfY1_s,how='left',on='GEO_ID')
dfY1 = get.clean_data(dfY1,var_data)

In [8]:
dfY1 = lf.make_lf_vars(dfY1,y1)
dfY1 = dfY1.drop(var_data[1:],axis=1)

In [9]:
#dfY1.head()

In [10]:
dfY0_p = get.get_county(p,source_p,y0,cols_p)
dfY0_s = get.get_county(p,source_s,y0,cols_s)
dfY0 = pd.merge(dfY0_p,dfY0_s,how='left',on='GEO_ID')
dfY0 = get.clean_data(dfY0,var_data)
#dfY1.head()

In [11]:
dfY0 = lf.make_lf_vars(dfY0,y0)
dfY0 = dfY0.drop(var_data[1:],axis=1)

In [12]:
#dfY0.head()

In [13]:
#Merge the two dataframes using the stco identifier
dfY0Y1 = pd.merge(dfY0,dfY1,how='left',on='GEO_ID')
dfY0Y1 = lf.make_lf_change(dfY0Y1,y0,y1)

In [14]:
dfY0Y1.head()

,GEO_ID,Pop_10E,Pop_10M,Pop_10C,LF_10E,LF_10M,LF_10C,Pop1624_10E,Pop1624_10M,Pop1624_10C,...,Pop4554_1020C,LF4554_1020E,LF4554_1020M,LF4554_1020C,PopO55_1020E,PopO55_1020M,PopO55_1020C,LFO55_1020E,LFO55_1020M,LFO55_1020C
0,09001,703812.0,616.0,0.053206,475682.0,2304.0,0.294442,96465.0,1372.730126,0.865066,...,1.239047,-5519.0,1315.389676,14.488657,59992.0,3411.628350,3.457025,39041.0,2903.404725,4.520857
1,09005,153468.0,245.0,0.097047,107531.0,846.0,0.478267,18671.0,550.672316,1.792915,...,1.494813,-6729.0,638.235850,5.765869,14407.0,1616.306283,6.819997,7247.0,1474.843721,12.371485
2,09009,686161.0,694.0,0.061485,465571.0,2465.0,0.321859,108450.0,1740.966111,0.975876,...,0.605910,-13963.0,1427.312510,6.214046,47365.0,3323.605271,4.265658,23387.0,2812.692305,7.311085
3,34003,717394.0,589.0,0.049910,475551.0,2700.0,0.345144,91762.0,1312.998096,0.869832,...,0.718833,-6659.0,1323.466660,12.081979,49448.0,3602.535774,4.428878,34408.0,3253.459236,5.748043
4,34013,607411.0,556.0,0.055645,399973.0,2710.0,0.411882,96204.0,1417.450528,0.895672,...,8.805109,-303.0,1406.037339,282.090411,35644.0,3035.259462,5.176587,21637.0,2671.004493,7.504326


In [15]:
#Make subregion and region tables
subreg,reg = dfY0Y1.copy(),dfY0Y1.copy()

subreg['sub'],reg['reg']=subreg.GEO_ID.map(sub_5),'METRO'

subreg,reg = subreg.drop(columns='GEO_ID'),reg.drop(columns='GEO_ID')

subreg = subreg[['sub']+[col for col in subreg.columns if col != 'sub']]
reg = reg[['reg']+[col for col in reg.columns if col != 'reg']]

subreg,reg = geo_agg.sumgeo_cv(subreg,'sub'),geo_agg.sumgeo_cv(reg,'reg')

In [18]:
subreg = subreg.rename(columns={'sub':'GEO_ID'})
reg = reg.rename(columns={'reg':'GEO_ID'})

In [19]:
cosubreg = pd.concat([dfY0Y1,subreg,reg])

In [21]:
cosubreg.to_excel('../../output/Economy/ACS/ACS_LF_20102020.xlsx',index=False)